In [15]:
import numpy as np
import matplotlib.pyplot as plt
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)
# ====================================================================================
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
iris_dataset = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris_dataset.data,iris_dataset.target,
                                                   test_size=1/3,stratify=iris_dataset.target,
                                                   random_state=0)
X_train = torch.FloatTensor(X_train)
y_train = torch.LongTensor(y_train)
X_test = torch.FloatTensor(X_test)
y_test = torch.LongTensor(y_test)
# y_train one-hot incoding
y_one_hot = torch.zeros(100,3)
y_one_hot.scatter_(1,y_train.unsqueeze(1),1)
y_one_hot_test = torch.zeros(50,3)
y_one_hot_test.scatter_(1,y_test.unsqueeze(1),1)
# modeling
model = nn.Sequential().to(device)
model.add_module('fc1',nn.Linear(4,100)) # input layer
model.add_module('relu1',nn.ReLU())
model.add_module('fc2',nn.Linear(100,100)) # hidden layer1
model.add_module('relu2',nn.ReLU())
model.add_module('fc3',nn.Linear(100,100))
model.add_module('relu3',nn.ReLU())
model.add_module('fc4',nn.Linear(100,3))

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.01)

nb_epochs = 1000
for epoch in range(nb_epochs+1):
    
    prediction = model(X_train)
    loss = loss_fn(prediction,y_train)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('Epoch {:4d}/{} loss: {:.6f}'.format(
            epoch, nb_epochs, loss.item()
        ))

Epoch    0/1000 loss: 1.148980
Epoch  100/1000 loss: 0.061950
Epoch  200/1000 loss: 0.051860
Epoch  300/1000 loss: 0.050612
Epoch  400/1000 loss: 0.048469
Epoch  500/1000 loss: 0.170972
Epoch  600/1000 loss: 0.042345
Epoch  700/1000 loss: 0.050885
Epoch  800/1000 loss: 0.033196
Epoch  900/1000 loss: 0.060994
Epoch 1000/1000 loss: 0.041424


In [16]:
#model testing
with torch.no_grad():
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())


Accuracy: 0.9599999785423279
